In [11]:
from __future__ import annotations  #Use to set the type of function output. eg. def abc(a:bool,b:bool)->bool is simailar to bool abc(bool a , bool b) in c++
from math import ceil, floor        #Defin ceil and floor function to help split and merge operation in B+ tree
import time
#Speically, for B+tree and B tree, plrease use order≥3
#This is the most important part and difficult part in CA2.
#The explanation below shows the meaning of decorator @staticmethod.
'''
About @staticmethod
A static method does not receive an implicit first argument. 
To declare a static method, use this idiom:
     class C:  
         @staticmethod  
         def f(arg1, arg2, ...):  
             ...  
It can be called either on the class (e.g. C.f()) or on an instance (e.g. C().f()). 
The instance is ignored except for its class.
'''

#Exercise1 Relation
class Relation:
    class smallBlock:
        def __init__(self,size):
            self.size = size
            self.tuples = [None] * size
            self.numtuples = 0
        def __getitem__(self,idx):
            return self.tuples[idx]
        def __setitem__(self,idx,t):
            self.tuples[idx] = t
        def getnumtuples(self):
            return self.numtuples
        def insert(self,t):
            for i in range(self.size):
                if self.tuples[i] == None:
                    self.tuples[i] = t
                    self.numtuples += 1
                    return i
            raise IndexError("The block is full.")

        def retrieve(self,idx):
            return self.tuples[idx]
        
        def isfull(self):
            count=0
            for i in range(self.size):
                if self.tuples[i] != None:
                    count +=1
            if count==self.size-1:
                return True
            return False

    class __Placeholder:
        def __init__(self):
            pass
        def __eq__(self,other):
            return False

    def __init__(self,thesize,key):
        self.size=thesize
        self.elements=[None]*self.size
        self.key=key
        for i in range(0,self.size//2):
            #The initialization part of the function. The first block is for overflow block, the second one is main data block.
            self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]  

    def insert(self,t):
        idx1=t[0]%40
        idx2=t[0]//40
        if self.elements[idx2][0].isfull()== False:
            self.elements[idx2][0][idx1]=t
            self.elements[idx2][0].numtuples+=1

        if self.elements[idx2][0].isfull()== True:
            self.sorting(idx2)
            self.split(self.elements[idx2][1],idx2)
        return self.findpointer(t)
        
    
    def deletion(self,t):
        flag=0
        for i in self.elements:
            if i != None:
                for j in i[0].tuples:
                    if t==j:
                        flag=1
                        num=i[0].tuples.index(j)
                        i[0].tuples[num]=Relation.__Placeholder()
                        i[0].numtuples-=1
                        if i[0].numtuples ==i[0].size//2:
                            self.merge(i)
                        return

                for k in i[1].tuples:
                    if t==k:
                        flag=1
                        num=i[1].tuples.index(k)
                        i[1].tuples[num]=Relation.__Placeholder()
                        i[1].numtuples-=1
                        if i[1].numtuples ==i[1].size//2:
                            self.merge(i)
                        return
        if flag==1:
            return True
            
        if flag==0:
            raise IndexError('No such key')
    
    def keydelete(self,key):
        element=self.find(key)
        self.deletion(element)

    def sorting(self,idx2):
        list0=[]
        for i in self.elements[idx2][0].tuples:
            if i!=None and type(i)!=Relation.__Placeholder:
                list0.append(i)
        for i in self.elements[idx2][1].tuples:
            if i!=None and type(i)!=Relation.__Placeholder:
                list0.append(i)

        #Lots of differnet keys
        if self.key=='Movie':
            list0.sort(key=lambda x:(x[1],x[2]))

        if self.key=='Person':
            list0.sort(key=lambda x:x[1])
        
        if self.key=='Award':
            list0.sort(key=lambda x:x[1])
        
        if self.key=='Restriction_Category':
            list0.sort(key=lambda x:(x[1],x[2]))
        
        if self.key=='Director':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Writer':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Crew':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Role':
            list0.sort(key=lambda x:(x[2],x[3],x[4]))
        
        if self.key=='Scene':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Restriction':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Appearance':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Movie_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Crew_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Director_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Writer_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Actor_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))

        if self.key=='QMovie_Award':
            list0.sort(key=lambda x:(list(x[2].values())[0][2],list(x[2].values())[1][2],list(x[2].values())[2][2]))

        if self.key=='QPerson':
            list0.sort(key=lambda x:list(x[2].values())[0][2])
        
        if self.key=='QRole':
            list0.sort(key=lambda x:(list(x[2].values())[1][2],list(x[2].values())[2][2],list(x[2].values())[3][2]))
        
        if self.key=='QScene':
            list0.sort(key=lambda x:(list(x[2].values())[0][2],list(x[2].values())[1][2],list(x[2].values())[2][2]))
        
        if self.key=='QAppearance':
            list0.sort(key=lambda x:(list(x[2].values())[0][2],list(x[2].values())[1][2],list(x[2].values())[2][2],list(x[2].values())[3][2]))

        self.elements[idx2][0]=Relation.smallBlock(40)
        self.elements[idx2][1]=Relation.smallBlock(40)
        for j in list0:
            self.elements[idx2][1].insert(j)
        
    
    def split(self,elements,idx):
        for i in range(self.size//2,self.size):
            if self.elements[i]==None:
                self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]
                for j in range(self.elements[i][0].size//2,self.elements[i][0].size):
                    self.elements[i][0].insert(elements[j])
                    self.elements[idx][1][j]=None
                    self.elements[idx][1].numtuples-=1
                return i 
                    


    
    def merge(self,tuplea):
        list1=[]
        for i in tuplea[0].tuples:
            if i!= None:
                list1.append(i)
        for i in tuplea[1].tuples:
            if i!= None:
                list1.append(i)
        for i in range(0,self.size):
            if self.elements[i]==tuplea:
                if i==0:
                    for k in list1:
                        self.elements[1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]
                    return
                if i==self.size-1:
                    for k in list1:
                        self.elements[self.size-1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]
                    return
                if self.elements[i-1][1].numtuples>=self.elements[i+1][1].numtuples:
                    for k in list1:
                        self.elements[i+1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]
                    return
                if self.elements[i-1][1].numtuples < self.elements[i+1][1].numtuples:
                    for k in list1:
                        self.elements[i-1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(40),Relation.smallBlock(40)]
                    return

    #This function is same as find/ another kind of retireval.
    def find(self,num):
        #Linear search
        for i in self.elements:
            if i != None:
                for j in i[0].tuples:
                    if j!=None and type(j)!=Relation.__Placeholder and num == j[0]:
                        return j
                couplelist=[]
                for j in i[1].tuples:
                    if j!= None and type(j)!=Relation.__Placeholder:
                        couplelist.append(j)
                if couplelist ==[]:
                    continue
                couplelist.sort(key=lambda x:x[0])
                left=0
                right=len(couplelist)
                #binary search
                while left<=right:
                    mid=(left+right)//2
                    if num<couplelist[mid-1][0]:
                        right=mid-1
                    elif num>couplelist[mid-1][0]:
                        left=mid+1
                    else:
                        return couplelist[mid-1]
        return -1
    
    def retrievedirectly(self,pointer):
        blocknumber = pointer[0]
        idx = pointer[1]
        blocktype=pointer[2]
        return self.elements[blocknumber][blocktype][idx]
    
    def findpointer(self,t):
        for i in range(len(self.elements)):
            if self.elements[i] != None:
                for j in range(self.elements[i][0].size):
                    if self.elements[i][0][j]==t:
                        return (i,j,0)
                for j in range(self.elements[i][1].size):
                    if self.elements[i][1][j]==t:
                        return (i,j,1)




In [12]:
#Node class for B + tree.
class Node:
    #Use to debug and show structure
    uid_counter=0
    def __init__(self, order):
        self.order = order
        self.parent: Node = None
        self.keys = []
        self.pointerlist = []
        #For debug and show structure
        Node.uid_counter += 1
        self.uid = self.uid_counter
    
    def split(self) -> Node:  # Split a full Node to two new ones.
        left = Node(self.order)
        right = Node(self.order)
        mid = int(self.order // 2)

        left.parent = right.parent = self

        left.keys = self.keys[:mid]
        left.pointerlist = self.pointerlist[:mid+1]

        right.keys = self.keys[mid+1:]
        right.pointerlist = self.pointerlist[mid+1:]

        self.pointerlist = [left, right]  # set up the pointers to child nodes.

        self.keys = [self.keys[mid]]  # we hold the first element from the right subtree.

        # This step is to set up correct parent for each child node.
        #isinstance(a,b) is same as 'type(a)==b'
        for child in left.pointerlist:
            if isinstance(child, Node):
                child.parent = left

        for child in right.pointerlist:
            if isinstance(child, Node):
                child.parent = right

        return self  
    def get_size(self) -> int:
        return len(self.keys)

    def is_empty(self) -> bool:
        return len(self.keys) == 0

    def is_full(self) -> bool:
        return len(self.keys) == self.order - 1

    # Used to check on keys, not data.
    def is_nearly_underflowed(self) -> bool:  
        return len(self.keys) <= floor(self.order / 2)

    # Used to check on keys, not data.
    def is_underflowed(self) -> bool:  
        return len(self.keys) <= floor(self.order / 2) - 1

    def is_root(self) -> bool:
        return self.parent is None

# LeafNode should inherit class node.
class LeafNode(Node):

    def __init__(self, order):
        super().__init__(order)
        self.prev_leaf = None
        self.next_leaf = None

    def add(self, key, value):  
         # Insert key, if the key doesn't exist
        if not self.keys: 
            self.keys.append(key)
            self.pointerlist.append([value])
            return

        #All keys mentioned below are primary key.
        # Otherwise, search primary key key and append the pointer.
        for i, item in enumerate(self.keys):  
            if key == item:  
                self.pointerlist[i].append(value)  
                break
            
             # If Key des not be found and key[0] < item[0], we need add key before item.
            elif key[0] < item[0]:  
                self.keys = self.keys[:i] + [key] + self.keys[i:]
                self.pointerlist = self.pointerlist[:i] + [[value]] + self.pointerlist[i:]
                break
            # If key does not be found here, we need to append it after.

            elif i + 1 == len(self.keys):  
                self.keys.append(key)
                self.pointerlist.append([value])
                break

    #This part is to split leaf node, which is different from the method used before.
    def split(self) -> Node: 
        top = Node(self.order)
        right = LeafNode(self.order)
        mid = int(self.order // 2)

        self.parent = right.parent = top

        right.keys = self.keys[mid:]
        right.pointerlist = self.pointerlist[mid:]
        right.prev_leaf = self
        right.next_leaf = self.next_leaf

        top.keys = [right.keys[0]]
        top.pointerlist = [self, right]  # We can set up the pointers to child nodes.

        self.keys = self.keys[:mid]
        self.pointerlist = self.pointerlist[:mid]
        self.next_leaf = right  # We cam set up the pointer to next leaf.

        return top  # Return the top node.

#This is main part for B + tree.
class BPlusTree(object):
    def __init__(self, order=5):
        #Clearly, the first node cannot be leaf.
        self.root  = LeafNode(order) 
        self.order = order

    @staticmethod
    #Find operation
    def _find(node: Node, key):
        for i, item in enumerate(node.keys):
            if key[0] < item[0]:
                return node.pointerlist[i], i
            elif i + 1 == len(node.keys):
                return node.pointerlist[i + 1], i + 1  # return right-most node and pointer.

    
    @staticmethod
    #This is merge operation, which for small size node.
    def _merge_up(parent: Node, child: Node, index):
        parent.pointerlist.pop(index)
        pivot = child.keys[0]

        for c in child.pointerlist:
            if isinstance(c, Node):
                c.parent = parent

        for i, item in enumerate(parent.keys):
            if pivot[0] < item[0]:
                parent.keys = parent.keys[:i] + [pivot] + parent.keys[i:]
                parent.pointerlist = parent.pointerlist[:i] + child.pointerlist + parent.pointerlist[i:]
                break

            elif i + 1 == len(parent.keys):
                parent.keys += [pivot]
                parent.pointerlist += child.pointerlist
                break

    #Insert operation
    def insert(self, key, value, relation):
        node = self.root
        while not isinstance(node, LeafNode):  # While we are in internal nodes, we need to search for leafs.
            node, index = self._find(node, key)

        # Node is now set as one LeafNode!
        node.add(key, value[0])
        
        while len(node.keys) == node.order:  # over full
            if not node.is_root():
                parent = node.parent
                node = node.split()  # Split and Set node as the 'top' node.
                jnk, index = self._find(parent, node.keys[0])
                self._merge_up(parent, node, index)
                node = parent
            else:
                node = node.split()  # Split and Set node as the 'top' node.
                self.root = node  # To reassign operation
    
    #Retrieve method, to find key.
    def retrieve(self, key, relation):
        node = self.root
        while not isinstance(node, LeafNode):
            node, index = self._find(node, key) #While we are in internal nodes, we need to search for leafs.

        #Using enumerate method to implement the iteration for keys.
        for i, item in enumerate(node.keys):
            if key == item:
                #Output the data we found
                for k in node.pointerlist[i]:
                    set0=set(item)
                    set1=set(relation.find(k))
                    if set0.issubset(set1):
                        return relation.find(k)
        return None

    #Delete operation
    def delete(self, key, relation):
        node = self.root
        #Clearly, we need to delte the key both in leaf and node.
        while not isinstance(node, LeafNode):
            node, parent_index = self._find(node, key)
            if key in node.keys and isinstance(node,LeafNode)==False:
                index=node.keys.index(key)
                node.keys.remove(key)
                node.pointerlist.pop(index)
        if key not in node.keys:
            return False

        index = node.keys.index(key)
        node.pointerlist[index].pop()  # Remove the last inserted data.

        if len(node.pointerlist[index]) == 0:
            node.pointerlist.pop(index)  # Remove the list element.
            node.keys.pop(index)

            while node.is_underflowed() and not node.is_root(): #Overflow and adjust steps.
                # Borrow attempt:
                prev_sibling = BPlusTree.get_prev_sibling(node)
                next_sibling = BPlusTree.get_next_sibling(node)
                jnk, parent_index = self._find(node.parent, key)

                if prev_sibling and not prev_sibling.is_nearly_underflowed():
                    self._borrow_left(node, prev_sibling, parent_index)
                elif next_sibling and not next_sibling.is_nearly_underflowed():
                    self._borrow_right(node, next_sibling, parent_index)
                elif prev_sibling and prev_sibling.is_nearly_underflowed():
                    self._merge_on_delete(prev_sibling, node)
                elif next_sibling and next_sibling.is_nearly_underflowed():
                    self._merge_on_delete(node, next_sibling)
                node = node.parent

            if node.is_root() and not isinstance(node, LeafNode) and len(node.pointerlist) == 1:
                self.root = node.pointerlist[0]
                self.root.parent = None

    @staticmethod
    #Help to reorganize
    def _borrow_left(node: Node, sibling: Node, parent_index):
        if isinstance(node, LeafNode):  # To check redistribution of the leaf
            key = sibling.keys.pop(-1)
            data = sibling.pointerlist.pop(-1)
            node.keys.insert(0, key)
            node.pointerlist.insert(0, data)

            node.parent.keys[parent_index-1] = key  # This step is to update Parent situation.
        else:  #To check redistribution of the inner node.
            parent_key = node.parent.keys.pop(-1)
            sibling_key = sibling.keys.pop(-1)
            data: Node = sibling.pointerlist.pop(-1)
            data.parent = node

            node.parent.keys.insert(0, sibling_key)
            node.keys.insert(0, parent_key)
            node.pointerlist.insert(0, data)

    @staticmethod
    #Help to reorganize.
    def _borrow_right(node: LeafNode, sibling: LeafNode, parent_index):
        if isinstance(node, LeafNode):  # To check redistribution of the leaf
            key = sibling.keys.pop(0)
            data = sibling.pointerlist.pop(0)
            node.keys.append(key)
            node.pointerlist.append(data)
            node.parent.keys[parent_index] = sibling.keys[0]  #This step is to update Parent situation.
        else:   #To check redistribution of the inner node.
            parent_key = node.parent.keys.pop(0)
            sibling_key = sibling.keys.pop(0)
            data: Node = sibling.pointerlist.pop(0)
            data.parent = node

            node.parent.keys.append(sibling_key)
            node.keys.append(parent_key)
            node.pointerlist.append(data)

    @staticmethod
    #Merge operation(reorganization)
    #Merge two nodes
    def _merge_on_delete(l_node: Node, r_node: Node):
        parent = l_node.parent
        jnk, index = BPlusTree._find(parent, l_node.keys[0])  # This step is to reset the pointer to child
        parent_key = parent.keys.pop(index)
        parent.pointerlist.pop(index)
        parent.pointerlist[index] = l_node

        if isinstance(l_node, LeafNode) and isinstance(r_node, LeafNode):
            l_node.next_leaf = r_node.next_leaf  # This step is to change the next leaf pointer
        else:
            l_node.keys.append(parent_key)  # Verify it as a inner node, then adjust.
            for r_node_child in r_node.pointerlist:
                r_node_child.parent = l_node

        l_node.keys += r_node.keys
        l_node.pointerlist += r_node.pointerlist

    @staticmethod
    #Find siblings, similar as B tree.
    def get_prev_sibling(node: Node) -> Node:
        if node.is_root() or not node.keys:
            return None
        jnk, index = BPlusTree._find(node.parent, node.keys[0])
        return node.parent.pointerlist[index-1] if index-1 >= 0 else None

     
    @staticmethod
    #Find siblings, similar as B tree.
    def get_next_sibling(node: Node) -> Node:
        if node.is_root() or not node.keys:
            return None
        jnk, index = BPlusTree._find(node.parent, node.keys[0])
        return node.parent.pointerlist[index+1] if index+1 < len(node.parent.pointerlist) else None
    
    @staticmethod
    #Help to show the organize of data
    def enlargequeue(lst, item):
        result = [item] * (len(lst) * 2)
        result[0::2] = lst
        return result

    #show the key structure of b+ tree by BFS
    def show_bfs(self):
        if self.root.is_empty():
            print('The B+ Tree is empty!')
            return

        queue = [self.root, 0]  # Node, Height... basic steps for bfs.

        while len(queue) > 0:
            node = queue.pop(0)
            height = queue.pop(0)
            if not isinstance(node, LeafNode):
                queue += self.enlargequeue(node.pointerlist, height+1)
            print(height,':', '|'.join(map(str, node.keys)), '\t', '\t parent -> ', node.parent.keys if node.parent else None)

    #Create a key list for leaf nodes.
    def keys_bfs(self):
        list=[]
        if self.root.is_empty():
            print('The B+ Tree is empty!')
            return
        queue = [self.root, 0]  # Node, Height... Scrappy but it works
        while len(queue) > 0:
            node = queue.pop(0)
            height = queue.pop(0)
            if not isinstance(node, LeafNode):
                queue += self.enlargequeue(node.pointerlist, height+1)
            if isinstance(node,LeafNode):
                list.append(node.keys)
        return list

    #use this method to find a specific tuple.
    def keyfind(self,data):
        list=self.keys_bfs()
        for i in list:
            for j in i:
                set0=set(j)
                set1=set(data)
                if set0.issubset(set1):
                    return j

    #Show the orgnization of leaf node by linked list
    def show_all_data(self):
        node = self.get_leftmost_leaf()
        if not node:
            return None

        while node:
            for node_data in node.pointerlist:
                print('[{}]'.format(', '.join(map(str, node_data))), end=' -> ')
            node = node.next_leaf
        print('Last node')

    #Functions that help to show all data.
    #Use for the leftmost leaf.
    def get_leftmost_leaf(self):
        if not self.root:
            return None
        node = self.root
        while not isinstance(node, LeafNode):
            node = node.pointerlist[0]
        return node


In [13]:
#This is the part for B tree node implement in the class
#But we slightly change some operations in the original index structure.
class BTreeNode:
    def __init__(self,order):
        self.size = 3 * order
        self.items = [None] * self.size
        self[0] = 0

    #One help method to show the structure for b tree.
    def pr_print(self,db,number):
        print("-------------")
        print("Btree node #",number)
        print(self)
        print("number of Keys:",self.getnumKeys(),"Parent node:", self.getParent())
        print("Child 0:", self.getChild(0))
        for i in range(1,self.getnumKeys()+1):
            print("Key",i,":",self.getKey(i))
            print("Pointer",i,":",self.getData(i))
            print("Child",i,":",self.getChild(i))

    #A series of operations to change the node content, the structure of which is derived from the slides.
    def __setitem__(self,index,val):
        if index >= 0 and index < self.size:
            self.items[index] = val
            return
        raise IndexError("BTree node assignment index out of range")
    def __getitem__(self,index):
        if index >= 0 and index < self.size:
            return self.items[index]
        raise IndexError("Btree node index out of range")
    def getnumKeys(self):
        return self[0]
    def getParent(self):
        return self[1]
    def getChild(self,index):
        idx = 3 * index + 2
        if index >= 0 and index <= self[0]:
            return self[idx]
        raise IndexError("There is no child with this index")
    def getKey(self,index):
        idx = 3 * index
        if index >= 1 and index <= self.getnumKeys():
            return self[idx]
        raise IndexError("There is no key with this index")
    def getData(self,index):
        idx = 3 * index + 1
        if index >= 1 and index <= self[0]:
            return self[idx]
        raise IndexError("There is no key with this index")
    def setnumKeys(self,nokeys):
        self[0] = nokeys
    def setParent(self,parent):
        self[1] = parent
    def setChild(self,index,child):
        idx = 3 * index + 2
        if idx >= 2 and idx <= self.size:
            self[idx] = child
    def setKey(self,index,key):
        idx = 3 * index
        if idx >= 3 and idx <= self.size:
            self[idx] = key
    def setData(self,index,data):
        idx = 3 * index + 1
        if idx >= 4 and idx <= self.size:
            self[idx] = data
    def find(self,key):
        if self.getnumKeys() != 0:
            minidx, maxidx = 1, self.getnumKeys()
            return self.__find(key,minidx,maxidx)
        return (0,None)
    def __find(self,key,min,max):
        if min == max:
            if self.getKey(min) == key:
                return (min,None)
            if key < self.getKey(min):
                return (0,self.getChild(min-1))
            return (0,self.getChild(min))
        else:
            new = (min + max) // 2
            if self.getKey(new) == key:
                return (new,None)
            if key < self.getKey(new):
                return self.__find(key,min,new)
            else:
                return self.__find(key,new+1,max)
    def __locate(self,key,min,max):
        med = (min + max) // 2
        if key < self.getKey(med):
            if min == med:
                return min
            return self.__locate(key,min,med)
        if key > self.getKey(med):
            if max == med:
                return max + 1
            return self.__locate(key,med+1,max)
        raise IndexError("The key already exists.")
    def insertatIndex(self,idx,k,p,c):
        for i in range(self.getnumKeys(),idx-1,-1):
            self.setKey(i+1,self.getKey(i))
            self.setData(i+1,self.getData(i))
            self.setChild(i+1,self.getChild(i))
        self.setKey(idx,k)
        self.setData(idx,p)
        self.setChild(idx,c)
        self.setnumKeys(self.getnumKeys()+1)

    def insert(self,k,p,c):
        if 3 * (self.getnumKeys() + 1) < self.size:
            if self.getnumKeys() == 0:
                idx = 1
            else:
                min, max = 1, self.getnumKeys()
                idx = self.__locate(k,min,max)
            self.insertatIndex(idx,k,p,c)
            return None
        separator = self.split(k,p,c)
        parent = self.getParent()
        if parent != None:
            return parent.insert(separator[0],separator[1],separator[2])
        order = self.size // 3
        newroot = BTreeNode(order)
        newroot.setnumKeys(1)
        newroot.setChild(0,self)
        newroot.setKey(1,separator[0])
        newroot.setData(1,separator[1])
        newroot.setChild(1,separator[2])
        self.setParent(newroot)
        separator[2].setParent(newroot)
        return newroot

    #Split operation, for one nearly full node.
    def split(self,key,p,c):
        min, max = 1, self.getnumKeys()
        idx = self.__locate(key,min,max)
        halflength = (self.getnumKeys() + 1) // 2
        order = self.size // 3
        newnode = BTreeNode(order)
        newnode.setnumKeys(self.getnumKeys() - halflength)
        newnode.setParent(self.getParent())
        key_idx = 1
        for i in range(1,self.getnumKeys()+1):
            if self.getKey(i) < key:
                key_idx += 1
        if key_idx <= halflength:
            newkey = self.getKey(halflength)
            newdata = self.getData(halflength)
            newnode.setChild(0,self.getChild(halflength))
            for i in range(halflength,key_idx+1,-1):
                self.setKey(i,self.getKey(i-1))
                self.setData(i,self.getData(i-1))
                self.setChild(i,self.getChild(i-1))
            self.setKey(key_idx,key)
            self.setData(key_idx,p)
            self.setChild(key_idx,c)
            for i in range(1,newnode.getnumKeys()+1):
                j = i + halflength
                newnode.setKey(i,self.getKey(j))
                newnode.setData(i,self.getData(j))
                newnode.setChild(i,self.getChild(j))           
        else:
            if key_idx == halflength + 1:
                newkey = key
                newdata = p
                newnode.setChild(0,c)
                for i in range(1,newnode.getnumKeys()+1):
                    j = i + halflength
                    newnode.setKey(i,self.getKey(j))
                    newnode.setData(i,self.getData(j))
                    newnode.setChild(i,self.getChild(j))
            else:
                newkey = self.getKey(halflength+1)
                newdata = self.getData(halflength+1)
                newnode.setChild(0,self.getChild(halflength+1))
                for i in range(1,newnode.getnumKeys()):
                    j = i + halflength + 1
                    key_idx = 1
                    if self.getKey(j) < key:
                        newnode.setKey(i,self.getKey(j))
                        newnode.setData(i,self.getData(j))
                        newnode.setChild(i,self.getChild(j))
                        key_idx += 1
                    else:
                        newnode.setKey(i,self.getKey(j-1))
                        newnode.setData(i,self.getData(j-1))
                        newnode.setChild(i,self.getChild(j-1))
                if key_idx != newnode.getnumKeys():
                    newnode.setKey(newnode.getnumKeys(),self.getKey(j))
                    newnode.setData(newnode.getnumKeys(),self.getData(j))
                    newnode.setChild(newnode.getnumKeys(),self.getChild(j))
                newnode.setKey(key_idx,key)
                newnode.setData(key_idx,p)
                newnode.setChild(key_idx,c)
        self.setnumKeys(halflength)

        for i in range(newnode.getnumKeys()+1):
            chld = newnode.getChild(i)
            if chld != None:
                chld.setParent(newnode)
        return(newkey,newdata,newnode)

    def leftsibling(self,node): #left brother
        if self.getChild(0) == node:
            return None
        for i in range(1,self.getnumKeys()+1):
            if self.getChild(i) == node:
                separator = i
                return (self.getChild(i-1),separator)

    def rightsibling(self,node):#right brother
        for i in range(self.getnumKeys()):
            if self.getChild(i) == node:
                separator = i+1
                return (self.getChild(i+1),separator)
        return None
    #Basic merge operation
    def __merge(node1,Key,node2):
        numitems = node1.getnumKeys() + node2.getnumKeys() + 1
        med = numitems // 2 + 1
        if node1.getnumKeys() < node2.getnumKeys():
            num1 = node1.getnumKeys()
            node1.setnumKeys(med-1)
            node1.setKey(num1+1,Key[0])
            node1.setData(num1+1,Key[1])
            node1.setChild(num1+1,node2.getChild(0))
            for i in range(num1+2,med):
                node1.setKey(i,node2.getKey(i-num1-1))
                node1.setData(i,node2.getData(i-num1-1))
                node1.setChild(i,node2.getChild(i-num1-1))
            k = node2.getKey(node2.getnumKeys()-med+1)
            p = node2.getData(node2.getnumKeys()-med+1)
            node2.setChild(0,node2.getChild(node2.getnumKeys()-med+1))
            num2 = numitems - med
            for i in range(1,num2+1):
                node2.setKey(i,node2.getKey(i+node2.getnumKeys()-med+1))
                node2.setData(i,node2.getData(i+node2.getnumKeys()-med+1))
                node2.setChild(i,node2.getChild(i+node2.getnumKeys()-med+1))
            node2.setnumKeys(num2)
            return (k,p)
        k = node1.getKey(med)
        p = node1.getData(med)
        num2 = numitems - med
        for i in range(node2.getnumKeys(),0,-1):
            node2.setKey(i+num2-node2.getnumKeys(),node2.getKey(i))
            node2.setData(i+num2-node2.getnumKeys(),node2.getData(i))
            node2.setChild(i+num2-node2.getnumKeys(),node2.getChild(i))
        node2.setKey(num2-node2.getnumKeys(),Key[0])
        node2.setData(num2-node2.getnumKeys(),Key[1])
        node2.setChild(num2-node2.getnumKeys(),node2.getChild(0))
        for i in range(1,num2-node2.getnumKeys()):
            node2.setKey(i,node1.getKey(i+med))
            node2.setData(i,node1.getData(i+med))
            node2.setChild(i,node1.getChild(i+med))
        node2.setChild(0,node1.getChild(med))
        node1.setnumKeys(med-1)
        node2.setnumKeys(num2)
        return (k,p)

    #Delete operation, for one specific index.
    def delete(self,index):
        for i in range(index,self.getnumKeys()):
            self.setKey(i,self.getKey(i+1))
            self.setData(i,self.getData(i+1))
            self.setChild(i,self.getChild(i+1))
        self.setnumKeys(self.getnumKeys()-1)
        parent = self.getParent()
        left = parent.leftsibling(self)
        right = parent.rightsibling(self)
        if left != None:
            leftSibling = left[0]
            sepleft = left[1]
        else:
            leftSibling = None
        if right != None:
            rightSibling = right[0]
            sepright = right[1]
        else:
            rightSibling = None
        order = self.size // 3
        if self.getnumKeys() < order // 2:
            if leftSibling != None and leftSibling.getnumKeys() > order // 2:
                separator = (parent.getKey(sepleft),parent.getData(sepleft))
                newKey = BTreeNode.__merge(leftSibling,separator,self)
                parent.setKey(sepleft,newKey[0])
                parent.setData(sepleft,newKey[1])
                return None
            if rightSibling != None and rightSibling.getnumKeys() > order // 2:
                separator = (parent.getKey(sepright),parent.getData(sepright))
                newKey = BTreeNode.__merge(self,separator,rightSibling)
                parent.setKey(sepright,newKey[0])
                parent.setData(sepright,newKey[1])
                return None
            if leftSibling != None:
                separator = (parent.getKey(sepleft),parent.getData(sepleft))
                newnumKeys = leftSibling.getnumKeys()+self.getnumKeys()+1
                for i in range(1,self.getnumKeys()+1):
                    leftSibling.setKey(i+leftSibling.getnumKeys()+1,self.getKey(i))
                    leftSibling.setData(i+leftSibling.getnumKeys()+1,self.getData(i))
                    leftSibling.setChild(i+leftSibling.getnumKeys()+1,self.getChild(i))
                leftSibling.setKey(leftSibling.getnumKeys()+1,separator[0])
                leftSibling.setData(leftSibling.getnumKeys()+1,separator[1])
                leftSibling.setChild(leftSibling.getnumKeys()+1,self.getChild(0))
                leftSibling.setnumKeys(newnumKeys)
                if parent.getnumKeys() > 1:
                    return parent.delete(sepleft)
                return leftSibling
            separator = (parent.getKey(sepright),parent.getData(sepright))
            newnumKeys = rightSibling.getnumKeys()+self.getnumKeys()+1
            for i in range(1,rightSibling.getnumKeys()+1):
                self.setKey(i+self.getnumKeys()+1,rightSibling.getKey(i))
                self.setData(i+self.getnumKeys()+1,rightSibling.getData(i))
                self.setChild(i+self.getnumKeys()+1,rightSibling.getChild(i))
            self.setKey(self.getnumKeys()+1,separator[0])
            self.setData(self.getnumKeys()+1,separator[1])
            self.setChild(self.getnumKeys()+1,rightSibling.getChild(0))
            self.setnumKeys(newnumKeys)
            if parent.getnumKeys() > 1:
                return parent.delete(sepright)
            self.setParent(None)
            return self
        return None
            
#This is the main implement part for B tree.
class BTree:
    def __init__(self,db,order):
        self.main = db
        self.root = BTreeNode(order)
    #This method helps to show the structure for B tree.
    def prprint(self):
        BTree.__ppt(self.root,self.main,"0")

    def __ppt(node,db,num):
        if node != None:
            node.pr_print(db,num)
            for i in range(node.getnumKeys()+1):
                numext = num+"."+str(i)
                BTree.__ppt(node.getChild(i),db,numext)

    def find(self,key):
        return BTree.__find(self.root,self.main,key)
        
    def __find(node,db,key):
        location = node.find(key)
        idx = location[0]
        if idx != 0:
            pointer = node.getData(idx)
            record = db.retrievedirectly(pointer)
            return record
        child = location[1]
        if child != None:
            return BTree.__find(child,db,key)
        raise IndexError("There is no tuple with this key.")

    def __check(node,key):
        location = node.find(key)
        idx = location[0]
        if idx != 0:
            raise ValueError("There exists already a tuple with this key.")
        else:
            child = location[1]
            if child != None:
                return BTree.__check(child,key)
            return node

    #Only insert pointers and secondary key.
    def insert(self,key,t):
        leaf = BTree.__check(self.root,key)
        pointer = self.main.findpointer(t)
        res = leaf.insert(key,pointer,None)
        if res != None:
            self.root = res

    def __finddelete(node,db,key):
        location = node.find(key)
        idx = location[0]
        if idx != 0:
            pointer = node.getData(idx)
            return (node,idx)
        child = location[1]
        if child != None:
            return BTree.__finddelete(child,db,key)
        raise IndexError("There is no tuple with this key.")

    def __swapMax(node,ancestor,index):
        lastidx = node.getnumKeys()
        lastChild = node.getChild(lastidx)
        if lastChild != None:
            return BTree.__swapMax(lastChild,ancestor,index)
        kmax = node.getKey(lastidx)
        pmax = node.getData(lastidx)
        ancestor.setKey(index,kmax)
        ancestor.setData(index,pmax)
        return (node,lastidx)
        
    def delete(self,key):
        location = BTree.__finddelete(self.root,self.main,key)
        if location[0].getChild(location[1]) != None:
            leftchild = location[0].getChild(location[1]-1)
            location = BTree.__swapMax(leftchild,location[0],location[1])
        res = location[0].delete(location[1])
        if res != None:
            self.root = res


#Special insert operation for Bplus tree & B tree index structure.
def magicinsert(key,data,relation,btree,bptree):
    relation.insert(data)
    bptree.insert(key,data,relation)
    pointer=relation.findpointer(data)
    btree.insert(data[0],data)
    return pointer

#Special insert operation for Bplus tree & B tree index structure. 
#It works for primary key or secondary key.
def magicdelete(key,relation,btree,bptree):
    if type(key)==tuple:
        data=bptree.retrieve(key,relation)
        relation.deletion(data)
        bptree.delete(key,data)
        btree.delete(data[0])
    if type(key)==int:
        data=relation.find(key)
        primkey=bptree.keyfind(data)
        relation.deletion(data)
        bptree.delete(primkey,data)
        btree.delete(data[0])

#Special find operation
def magicfind(key,relation,btree,bptree):
    if type(key)==tuple:
        data=bptree.retrieve(key,relation)
        return data
    if type(key)==int:
        data=btree.find(key)
        return data

#Always call this function after every insert/delete operation set.
def reorganization(relation,btree,bptree):
    newbtree=BTree(relation,5)
    naivekeylist=bptree.keys_bfs()
    keylist=set()
    for i in naivekeylist:
        for j in i:
            keylist.add(j)
    keylist=list(keylist)
    keylist.sort()
    
    for m in keylist:
        element=bptree.retrieve(m,relation)
        newbtree.insert(element[0],element)
    return newbtree


In [14]:
#test
Movie=Relation(100,'Movie')
print('Initializing B+ tree and B tree...')
#Create structure
bplustree = BPlusTree(order=3)
btree = BTree(Movie,5)
print('test insert time')
beg=time.time()
magicinsert(('Titanic',1997),(1,'Titanic',1997,'USA',195,'romance'),Movie,btree,bplustree)
end=time.time()
print('time cost', end-beg)
magicinsert(('Shakespeare in Love',1998),(2,'Shakespeare in Love',1998,'UK',122,'romance'),Movie,btree,bplustree)
magicinsert(('The Cider House Rules',1999),(3,'The Cider House Rules',1999,'USA',125,'drama'),Movie,btree,bplustree)
magicinsert(('Gandhi',1982),(4,'Gandhi',1982,'India',188,'drama'),Movie,btree,bplustree)
magicinsert(('American Beauty',1999),(5,'American Beauty',1999,'USA',121,'drama'),Movie,btree,bplustree)
magicinsert(('Affliction',1997),(6,'Affliction',1997,'USA',113,'drama'),Movie,btree,bplustree)
magicinsert(('Life is Beautiful',1997),(7,'Life is Beautiful',1997,'Italy',118,'comedy'),Movie,btree,bplustree)
magicinsert(('Boys Dont Cry',1999),(8,'Boys Dont Cry',1999,'USA',118,'drama'),Movie,btree,bplustree)
magicinsert(('Saving Private Ryan',1998),(9,'Saving Private Ryan',1998,'USA',170,'action'),Movie,btree,bplustree)
magicinsert(('The Birds',1963),(10,'The Birds',1963,'USA',119,'horror'),Movie,btree,bplustree)
magicinsert(('The Matrix',1999),(11,'The Matrix',1999,'USA',136,'action'),Movie,btree,bplustree)
magicinsert(('Toy Story',1995),(12,'Toy Story',1995,'USA',81,'animation'),Movie,btree,bplustree)
magicinsert(('You have Got Mail',1998),(13,'You have Got Mail',1998,'USA',119,'comedy'),Movie,btree,bplustree)
magicinsert(('Proof of Life',2000),(14,'Proof of Life',2000,'USA',135,'drama'),Movie,btree,bplustree)
magicinsert(('Hanging Up',2000),(15,'Hanging Up',2000,'USA',94,'comedy'),Movie,btree,bplustree)
magicinsert(('The Price of Milk',2000),(16,'The Price of Milk',2000,'New Zealand',87,'romance'),Movie,btree,bplustree)
magicinsert(('The Footstep Man',1992),(17,'The Footstep Man',1992,'New Zealand',89,'drama'),Movie,btree,bplustree)
magicinsert(('Topless Women Talk About Their Lives',1997),(18,'Topless Women Talk About Their Lives',1997,'New Zealand',87,'drama'),Movie,btree,bplustree)
magicinsert(('The Piano',1993),(19,'The Piano',1993,'New Zealand',121,'romance'),Movie,btree,bplustree)
magicinsert(('Mad Max',1979),(20,'Mad Max',1979,'Australia',88,'action'),Movie,btree,bplustree)

print('show b tree')
btree.prprint()
print('show b+ tree')
bplustree.show_bfs()
print('show b+ tree data')
bplustree.show_all_data()



Initializing B+ tree and B tree...
test insert time
time cost 0.0001583099365234375
show b tree
-------------
Btree node # 0
number of Keys: 1 Parent node: None
Child 0: <__main__.BTreeNode object at 0x110f41050>
Key 1 : 9
Pointer 1 : (0, 9, 0)
Child 1 : <__main__.BTreeNode object at 0x110f97310>
-------------
Btree node # 0.0
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f97790>
Child 0: <__main__.BTreeNode object at 0x110fc6a90>
Key 1 : 3
Pointer 1 : (0, 3, 0)
Child 1 : <__main__.BTreeNode object at 0x10fdb9910>
Key 2 : 6
Pointer 2 : (0, 6, 0)
Child 2 : <__main__.BTreeNode object at 0x110efba90>
-------------
Btree node # 0.0.0
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f41050>
Child 0: None
Key 1 : 1
Pointer 1 : (0, 1, 0)
Child 1 : None
Key 2 : 2
Pointer 2 : (0, 2, 0)
Child 2 : None
-------------
Btree node # 0.0.1
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f41050>
Child 0: None
Key 1 : 4
Pointer 1 : (0, 4, 0)
Child 1

In [15]:
#test
print('test primary key delete')
beg=time.time()
magicdelete(('Mad Max',1979),Movie,btree,bplustree)
end=time.time()
print('time cost', end-beg)
print('show b tree delete')
btree.prprint()

print('test find operation')
beg=time.time()
print(magicfind(4,Movie,btree,bplustree))
print(magicfind(('Gandhi',1982),Movie,btree,bplustree))
end=time.time()
print('time cost', end-beg)

magicinsert(('Mad Max',1979),(20,'Mad Max',1979,'Australia',88,'action'),Movie,btree,bplustree)
print('test secondary key delete')
beg=time.time()
magicdelete(20,Movie,btree,bplustree)
end=time.time()
print('time cost', end-beg)

magicinsert(('Mad Max',1979),(20,'Mad Max',1979,'Australia',88,'action'),Movie,btree,bplustree)
magicinsert(('Strictly Ballroom',1992),(21,'Strictly Ballroom',1992,'Australia',94,'comedy'),Movie,btree,bplustree)
magicinsert(('My Mother Frank',2000),(22,'My Mother Frank',2000,'Australia',95,'comedy'),Movie,btree,bplustree)
magicinsert(('American Psycho',2000),(23,'American Psycho',2000,'Canada',101,'horror'),Movie,btree,bplustree)
magicinsert(('Scream 2',1997),(24,'Scream 2',1997,'USA',116,'horror'),Movie,btree,bplustree)
magicinsert(('Scream 3',2000),(25,'Scream 3',2000,'USA',116,'horror'),Movie,btree,bplustree)
magicinsert(('Traffic',2000),(26,'Traffic',2000,'Germany',147,'crime'),Movie,btree,bplustree)
magicinsert(('Psycho',1960),(27,'Psycho',1960,'USA',109,'horror'),Movie,btree,bplustree)
magicinsert(('I Know What You Did Last Summer',1997),(28,'I Know What You Did Last Summer',1997,'USA',100,'horror'),Movie,btree,bplustree)
magicinsert(('Cruel Intentions',1999),(29,'Cruel Intentions',1999,'USA',95,'romance'),Movie,btree,bplustree)
magicinsert(('Wild Things',1998),(30,'Wild Things',1998,'USA',108,'crime'),Movie,btree,bplustree)
magicinsert(('Alien',1979),(31,'Alien',1979,'UK',117,'horror'),Movie,btree,bplustree)
magicinsert(('Aliens',1986),(32,'Aliens',1986,'USA',154,'action'),Movie,btree,bplustree)
magicinsert(('Alien 3',1992),(33,'Alien 3',1992,'USA',115,'action'),Movie,btree,bplustree)
magicinsert(('Alien: Resurrection',1997),(34,'Alien: Resurrection',1997,'USA',109,'action'),Movie,btree,bplustree)
magicinsert(('Gladiator',2000),(35,'Gladiator',2000,'UK',154,'drama'),Movie,btree,bplustree)
magicinsert(('The World Is Not Enough',1999),(36,'The World Is Not Enough',1999,'UK',127,'action'),Movie,btree,bplustree)
magicinsert(('Heat',1995),(37,'Heat',1995,'USA',171,'action'),Movie,btree,bplustree)
magicinsert(('American History X',1998),(38,'American History X',1998,'USA',119,'drama'),Movie,btree,bplustree)
magicinsert(('Fight Club',1999),(39,'Fight Club',1999,'Germany',139,'drama'),Movie,btree,bplustree)
magicinsert(('Out of Sight',1998),(40,'Out of Sight',1998,'USA',122,'comedy'),Movie,btree,bplustree)

print('######show organization of Btree######')
print('test reorganization time')
beg=time.time()
btree=reorganization(Movie,btree,bplustree)
end=time.time()
print('time cost', end-beg)
btree.prprint() 
print('######show organization of Bplustree######')
bplustree.show_bfs()

test primary key delete
time cost 0.0004131793975830078
show b tree delete
-------------
Btree node # 0
number of Keys: 1 Parent node: None
Child 0: <__main__.BTreeNode object at 0x110f41050>
Key 1 : 9
Pointer 1 : (0, 9, 0)
Child 1 : <__main__.BTreeNode object at 0x110f97310>
-------------
Btree node # 0.0
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f97790>
Child 0: <__main__.BTreeNode object at 0x110fc6a90>
Key 1 : 3
Pointer 1 : (0, 3, 0)
Child 1 : <__main__.BTreeNode object at 0x10fdb9910>
Key 2 : 6
Pointer 2 : (0, 6, 0)
Child 2 : <__main__.BTreeNode object at 0x110efba90>
-------------
Btree node # 0.0.0
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f41050>
Child 0: None
Key 1 : 1
Pointer 1 : (0, 1, 0)
Child 1 : None
Key 2 : 2
Pointer 2 : (0, 2, 0)
Child 2 : None
-------------
Btree node # 0.0.1
number of Keys: 2 Parent node: <__main__.BTreeNode object at 0x110f41050>
Child 0: None
Key 1 : 4
Pointer 1 : (0, 4, 0)
Child 1 : None
Key 2 : 5
Poi